### **This Notebook Performs a Bayesian Parameter Search to find the best Cleaning and Model Parameters.**

Import Libraries

In [1]:
import sys
sys.path.append('..')
from Cleaning_and_Evaluation import *
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, GroupKFold
import optuna

Read the spectral data

In [2]:
#df = pd.read_csv("../../data/exosomes.raw_spectrum_1.csv")
# df = pd.read_csv("../../data/raw_df_outliers_removed.csv")
df = pd.read_csv("../../data/exosomes.raw_spectrum_400-1800.csv")

In [3]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia


In [4]:
wavelength_df = prepare_wavelength_df(df, 'Absorbance')
wavelength_df.head()

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,SurID,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-00,1765.6628,1774.7809,1769.0302,1756.4220,1758.8690,1763.2300,1745.2285,1773.3534,1774.7166,1753.3281,...,1210.4993,1213.9619,1225.2153,1210.0010,1210.6858,1194.4679,1195.1451,1189.8683,201210-1,Normal
201210-1-01,1966.9930,1962.4237,1954.5616,1954.3228,1963.0917,1975.0807,1979.3162,1963.4561,1968.4587,1964.0000,...,1382.6973,1363.7004,1360.6210,1354.0477,1353.0381,1353.9978,1361.2426,1370.2874,201210-1,Normal
201210-1-02,2182.6694,2149.6565,2146.0227,2159.3459,2167.2910,2160.9861,2145.6575,2134.2004,2142.8303,2138.6309,...,1976.2070,1989.0183,1996.2838,1979.3507,1976.2002,1994.9839,1974.2030,1971.1880,201210-1,Normal
201210-1-03,2445.0837,2430.4973,2422.7927,2434.3433,2454.9700,2462.8245,2454.7007,2467.7329,2449.5161,2421.3474,...,1992.3817,2022.6331,2001.8311,2010.0946,2006.4933,2017.2891,2038.1699,2000.6475,201210-1,Normal
201210-1-04,2250.4536,2248.6235,2245.0984,2242.7173,2235.2803,2228.9585,2236.0095,2229.6091,2225.9231,2211.0359,...,2009.0385,1953.3303,1963.5698,1964.5299,1969.5634,1986.6266,1970.1484,2007.0848,201210-1,Normal


---

>#### **Use Optuna to Find the Best Cleaning and Model Parameters**

In [5]:
df

,SpecID,Seq,WaveNumber,Absorbance,SurID,Status
0,201210-1-00,293,400.22778,1765.6628,201210-1,Normal
1,201210-1-00,294,400.91116,1774.7809,201210-1,Normal
2,201210-1-00,295,401.59454,1769.0302,201210-1,Normal
3,201210-1-00,296,402.27789,1756.4220,201210-1,Normal
4,201210-1-00,297,402.96127,1758.8690,201210-1,Normal
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,1617.3926,210526-3,Hyperglycemia
6239201,210526-3-09,2338,1797.72200,1633.0911,210526-3,Hyperglycemia
6239202,210526-3-09,2339,1798.40550,1633.3076,210526-3,Hyperglycemia
6239203,210526-3-09,2340,1799.08890,1641.8665,210526-3,Hyperglycemia


The objective function outlines the parameters search.

In [6]:
def objective(trial):

    # Decide which cleaning steps to implement
    despike = trial.suggest_categorical('despike', [True, False])
    baseline_correct = trial.suggest_categorical('baseline_correct', [True, False])
    smoothing = trial.suggest_categorical('smoothing', [True, False])
    scaling = trial.suggest_categorical('scaling', [True, False])

    df_copy = df.copy()

    # Apply preprocessing based on suggested parameters
    if despike:
        despike_ma = trial.suggest_int('despike_ma', 5, 200)
        despike_threshold = trial.suggest_float('despike_threshold', 3, 10, step=0.1)
        df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: despike_group(x, ma=despike_ma, threshold = despike_threshold))

    if baseline_correct:
        lam = trial.suggest_categorical('lam', [10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**8, 10**9, 10**10, 10**11])
        p = trial.suggest_float('p', 0.001, 0.1, step=0.001)
        df_copy['Baseline'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: asls_baseline_correction(x, lam=lam, p=p))
        df_copy['Absorbance'] = df_copy['Absorbance'] - df_copy['Baseline']

    if smoothing:
        window_size = trial.suggest_int('window_size', 6, 251)
        poly_order = trial.suggest_int('poly_order', 1, 5)
        df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: savgol_filter(x, window_size, poly_order, deriv=0))

    if scaling:
        scaling_type = trial.suggest_categorical('scaling_type', ['normal', 'vector', 'snv'])
        if scaling_type == 'normal':
            df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: normalise(x))
        elif scaling_type == 'vector':
            df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: vector_normalise(x))
        else:
            df_copy['Absorbance'] = df_copy.groupby('SpecID')['Absorbance'].transform(lambda x: snv_normalise(x))

    # Prepare data for ML
    wavelength_df = prepare_wavelength_df(df_copy, 'Absorbance')
    X = wavelength_df.drop(['Status', 'SurID'], axis=1)
    y = wavelength_df['Status']
    groups = wavelength_df['SurID']

    # Suggest classifier type
    classifier_name = trial.suggest_categorical("classifier", ["ExtraTrees", "RandomForest", "SVC"])

    if classifier_name == "ExtraTrees":
        criterion = trial.suggest_categorical("et_criterion", ["gini", "entropy"])
        n_estimators = trial.suggest_int("et_n_estimators", 10, 100)
        max_depth_option = trial.suggest_categorical("et_max_depth_option", [None, "Specify"])
        max_depth = trial.suggest_int("et_max_depth", 2, 32, log=True) if max_depth_option == "Specify" else None
        classifier = ExtraTreesClassifier(random_state=1234, criterion=criterion, n_estimators=n_estimators, max_depth=max_depth)

    elif classifier_name == "RandomForest":
        criterion = trial.suggest_categorical("rf_criterion", ["gini", "entropy"])
        n_estimators = trial.suggest_int("rf_n_estimators", 10, 100)
        max_depth_option = trial.suggest_categorical("rf_max_depth_option", [None, "Specify"])
        max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True) if max_depth_option == "Specify" else None
        classifier = RandomForestClassifier(random_state=1234, criterion=criterion, n_estimators=n_estimators, max_depth=max_depth)
        
    else:
        classifier = SVC(
            C=trial.suggest_float("svc_c", 1e-10, 1e10, log=True),
            kernel=trial.suggest_categorical("svc_kernel", ["linear", "poly", "rbf", "sigmoid"]),
            gamma=trial.suggest_categorical("svc_gamma", ["scale", "auto"]),
            random_state=1234,
        )

    cv = GroupKFold(n_splits=10)

    # Perform cross-validation
    scores = cross_validate(classifier, X, y, groups=groups, cv=cv, scoring='accuracy')

    # Return the average accuracy across all folds
    return np.mean(scores['test_score'])


sampler = optuna.samplers.TPESampler(seed=1234)  # Make the sampler behave in a deterministic way.
study = optuna.create_study(direction='maximize', sampler=sampler)

#study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100, n_jobs=-1)
study.optimize(objective, n_trials=500, n_jobs=-1)

print(study.best_trial)

[I 2024-04-17 10:05:05,095] A new study created in memory with name: no-name-99acf814-a83f-4260-b29a-3f5ad313c774
[I 2024-04-17 10:06:25,670] Trial 9 finished with value: 0.45831746031746035 and parameters: {'despike': False, 'baseline_correct': False, 'smoothing': False, 'scaling': True, 'scaling_type': 'normal', 'classifier': 'ExtraTrees', 'et_criterion': 'gini', 'et_n_estimators': 77, 'et_max_depth_option': 'Specify', 'et_max_depth': 9}. Best is trial 9 with value: 0.45831746031746035.
[I 2024-04-17 10:06:52,127] Trial 2 finished with value: 0.5594444444444445 and parameters: {'despike': False, 'baseline_correct': False, 'smoothing': False, 'scaling': True, 'scaling_type': 'snv', 'classifier': 'ExtraTrees', 'et_criterion': 'gini', 'et_n_estimators': 85, 'et_max_depth_option': None}. Best is trial 2 with value: 0.5594444444444445.
[I 2024-04-17 10:06:54,267] Trial 1 finished with value: 0.3269206349206349 and parameters: {'despike': False, 'baseline_correct': False, 'smoothing': True

In [1]:
results_df = study.trials_dataframe(attrs=("number", "value", "params", "state"))
results_df.to_csv("../../data/studies/optuna_cleaning_and_model_parameters.csv")

NameError: name 'study' is not defined

In [ ]:
import joblib

joblib.dump(study, "../../data/studies/optuna_cleaning_and_model_parameters.pkl")

# loaded_study = joblib.load("../../data/studies/cleaning_study.pkl")
# print("Best trial until now:")
# print(" Value: ", loaded_study.best_trial.value)
# print(" Params: ")
# for key, value in loaded_study.best_trial.params.items():
#     print(f"    {key}: {value}")

In [ ]:
print(study.best_trial)

In [ ]:
from optuna.visualization import plot_optimization_history, plot_slice

plot_optimization_history(study)

In [ ]:
plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

Continue the study.

In [ ]:
# study.optimize(objective, n_trials=100, n_jobs=-1)

# print(study.best_trial)